## To scrape data from DBLP profile page

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
from copy import deepcopy

In [2]:
file_name="dblp_search_results.json"
with open(file_name,'r') as fd:
    in_df=json.load(fd)

In [3]:
rep_tags=set()

In [4]:
def fetch_all_info(r_tag):
    main_tag=r_tag.find(recursive=False)
    #print(main_tag.name)
    main_tag_name=main_tag.name
    curr_dict={}
    curr_dict['work_type']=main_tag_name
    '''for child_tag in main_tag.find_all(recursive=False):
        curr_tag_name=child_tag.name
        if curr_tag_name in curr_dict:
            rep_tags.add(curr_tag_name)
            if curr_tag_name=='author':
                if curr_tag_name not in curr_dict:
                    curr_dict[curr_tag_name]=[]
                curr_dict[curr_tag_name].append(child_tag.text)
            else:
                print("Very surprising ", curr_tag_name)
        else:
            if curr_tag_name=='author':
                curr_dict[curr_tag_name]=[child_tag.text]
            else:
                curr_dict[curr_tag_name]=child_tag.text'''
    

    for child_tag in main_tag.find_all(recursive=False):
        curr_tag_name=child_tag.name
        if curr_tag_name in curr_dict:
            rep_tags.add(curr_tag_name)
            if type(curr_dict[curr_tag_name])!=list:
                curr_dict[curr_tag_name]=list(curr_dict[curr_tag_name])
            curr_dict[curr_tag_name].append(child_tag.text)
        else:
            if curr_tag_name=='author':
                curr_dict[curr_tag_name]=[child_tag.text]
            else:
                curr_dict[curr_tag_name]=child_tag.text
                
    if 'ee' in curr_dict and type(curr_dict['ee'])==list:
        curr_dict['ee']="".join(curr_dict['ee'])
    return deepcopy(curr_dict)        

In [5]:
def save_result():
    with open("./parsed_xmls_dblp.json",'w') as fd:
        in_df['curr_error_list_while_xml_parse']=unlucky_list
        in_df['done_idx']=idx
        json.dump(in_df, fd, indent=4,default=str)

In [6]:


unlucky_list=[]

for idx, person in enumerate(in_df['final_result']):
    
    person['dblp_data']=None
    curr_link=person['url_use']
    
    if curr_link!=None:
        try:
            ##########################
            #print(curr_link)
            
            if "html" in curr_link:
                curr_link=curr_link.replace("html","xml")
            else:
                curr_link+=".xml"
            #############################
            #print(curr_link)
            #break
    
            page = requests.get(curr_link)
            soup = BeautifulSoup(page.text, 'html.parser')
            master_dict={}
            tag=soup.find("dblpperson")
            
            #############################################
            person_tag=tag.find("person")
            curr_dict={}
            for child_tag in person_tag.find_all(recursive=False):
                curr_tag_name=child_tag.name
                if curr_tag_name in curr_dict:
                    rep_tags.add(curr_tag_name)
                    if type(curr_dict[curr_tag_name])!=list:
                        curr_dict[curr_tag_name]=list(curr_dict[curr_tag_name])
                    curr_dict[curr_tag_name].append(child_tag.text)
                else:
                    if curr_tag_name=='author':
                        curr_dict[curr_tag_name]=[child_tag.text]
                    else:
                        #print(curr_tag_name)
                        curr_dict[curr_tag_name]=child_tag.text
            master_dict['person_details']=curr_dict
            ############################################
            r_tags=tag.find_all('r')
            master_dict['work_details']=[fetch_all_info(x) for x in r_tags]
            co_author_tag=tag.find("coauthors")
            #print(co_author_tag)
            co_list=co_author_tag.find_all(recursive=False)
            #print(len(co_list))
            master_dict['co_author_list']=[x.find("na").text for x in co_list]
            person['dblp_data']=deepcopy(master_dict)
            #print("person is ", person['dblp_data'])
            
            #in_df['final_result'][idx][1]['retrieved_data']=person[1]['retrieved_data']
        except Exception as E:
            unlucky_list.append((person['ire_person_name'],E))
            
    else:
        unlucky_list.append((person['ire_person_name'],"amibguity"))
    in_df['final_result'][idx]=deepcopy(person)
    if idx%200==0:
        print("Now")
        print(idx)
        print("Saving")
        #break
        save_result()

Now
0
Saving
Now
200
Saving
Now
400
Saving
Now
600
Saving
Now
800
Saving
Now
1000
Saving
Now
1200
Saving
Now
1400
Saving
Now
1600
Saving
Now
1800
Saving
Now
2000
Saving
Now
2200
Saving
Now
2400
Saving
Now
2600
Saving
Now
2800
Saving
Now
3000
Saving
Now
3200
Saving
Now
3400
Saving


In [7]:
in_df['curr_error_list_while_xml_parse']=unlucky_list

In [8]:
save_result()